In [1]:
from MZPackage.ModelBuilder import ModelBuilder
from MZPackage.Utilities import *

In [2]:
# Create ModelBuilder object
dataSet = loadData("normalized_data_set")
mb = ModelBuilder()

### PCA

In [ ]:
modelType = "pca"
trainingSet, _, validationSet = getReshapedDataSetThreeSplits(dataSet, modelType)
codeLayerSize = 40
[model, validatoinLoss, numOfEpochs,_] = mb.trainPca(codeLayerSize, trainingSet, trainingSet, validationSet, validationSet)
model.save('Trained_Model/pca.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1800)              3241800   
_________________________________________________________________
dense_1 (Dense)              (None, 40)                72040     
_________________________________________________________________
dense_2 (Dense)              (None, 1800)              73800     
Total params: 3,387,640
Trainable params: 3,387,640
Non-trainable params: 0
_________________________________________________________________
Train on 11060 samples, validate on 1382 samples
Epoch 1/1000
11060/11060 [==============================] - 1s 88us/sample - loss: 0.1128 - val_loss: 0.0238
Epoch 2/1000
11060/11060 [==============================] - 0s 28us/sample - loss: 0.0801 - val_loss: 0.0250
Epoch 3/1000
11060/11060 [==============================] - 0s 30us/sample - loss: 0.0664 - val_loss: 0.0234


11060/11060 [==============================] - 0s 30us/sample - loss: 9.4090e-04 - val_loss: 0.0040
Epoch 69/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 0.0014 - val_loss: 0.0039
Epoch 70/1000
11060/11060 [==============================] - 0s 24us/sample - loss: 0.0013 - val_loss: 0.0038
Epoch 71/1000
11060/11060 [==============================] - 0s 29us/sample - loss: 0.0011 - val_loss: 0.0037
Epoch 72/1000
11060/11060 [==============================] - 0s 30us/sample - loss: 9.4731e-04 - val_loss: 0.0036
Epoch 73/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 8.7719e-04 - val_loss: 0.0035
Epoch 74/1000
11060/11060 [==============================] - 0s 27us/sample - loss: 8.4225e-04 - val_loss: 0.0034
Epoch 75/1000
11060/11060 [==============================] - 0s 27us/sample - loss: 8.1116e-04 - val_loss: 0.0034
Epoch 76/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 7.8061e-04 - val_loss: 0.0033


Epoch 140/1000
11060/11060 [==============================] - 0s 27us/sample - loss: 4.0644e-04 - val_loss: 0.0012
Epoch 141/1000
11060/11060 [==============================] - 0s 28us/sample - loss: 4.0096e-04 - val_loss: 0.0012
Epoch 142/1000
11060/11060 [==============================] - 0s 25us/sample - loss: 3.9781e-04 - val_loss: 0.0012
Epoch 143/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 3.9438e-04 - val_loss: 0.0012
Epoch 144/1000
11060/11060 [==============================] - 0s 29us/sample - loss: 3.9017e-04 - val_loss: 0.0012
Epoch 145/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 3.8744e-04 - val_loss: 0.0012
Epoch 146/1000
11060/11060 [==============================] - 0s 27us/sample - loss: 3.8717e-04 - val_loss: 0.0012
Epoch 147/1000
11060/11060 [==============================] - 0s 25us/sample - loss: 3.9670e-04 - val_loss: 0.0012
Epoch 148/1000
11060/11060 [==============================] - 0s 26us/sample - l

11060/11060 [==============================] - 0s 24us/sample - loss: 3.1730e-04 - val_loss: 9.2430e-04
Epoch 211/1000
11060/11060 [==============================] - 0s 25us/sample - loss: 3.2230e-04 - val_loss: 9.2175e-04
Epoch 212/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 3.2918e-04 - val_loss: 9.2129e-04
Epoch 213/1000
11060/11060 [==============================] - 0s 28us/sample - loss: 3.4096e-04 - val_loss: 9.1487e-04
Epoch 214/1000
11060/11060 [==============================] - 0s 28us/sample - loss: 3.3409e-04 - val_loss: 9.1107e-04
Epoch 215/1000
11060/11060 [==============================] - 0s 24us/sample - loss: 3.2430e-04 - val_loss: 9.1165e-04
Epoch 216/1000
11060/11060 [==============================] - 0s 24us/sample - loss: 4.0139e-04 - val_loss: 9.2910e-04
Epoch 217/1000
11060/11060 [==============================] - 0s 25us/sample - loss: 6.2134e-04 - val_loss: 9.1194e-04
Epoch 218/1000
11060/11060 [==============================] - 0

Epoch 279/1000
11060/11060 [==============================] - 0s 25us/sample - loss: 2.6493e-04 - val_loss: 7.9122e-04
Epoch 280/1000
11060/11060 [==============================] - 0s 27us/sample - loss: 2.6118e-04 - val_loss: 7.8961e-04
Epoch 281/1000
11060/11060 [==============================] - 0s 25us/sample - loss: 2.5779e-04 - val_loss: 7.8740e-04
Epoch 282/1000
11060/11060 [==============================] - 0s 32us/sample - loss: 2.5499e-04 - val_loss: 7.8519e-04
Epoch 283/1000
11060/11060 [==============================] - 0s 23us/sample - loss: 2.5468e-04 - val_loss: 7.8472e-04
Epoch 284/1000
11060/11060 [==============================] - 0s 27us/sample - loss: 2.5412e-04 - val_loss: 7.8342e-04
Epoch 285/1000
11060/11060 [==============================] - 0s 25us/sample - loss: 2.5550e-04 - val_loss: 7.8185e-04
Epoch 286/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 2.5951e-04 - val_loss: 7.8050e-04
Epoch 287/1000
11060/11060 [====================

Epoch 348/1000
11060/11060 [==============================] - 0s 23us/sample - loss: 2.4563e-04 - val_loss: 6.5359e-04
Epoch 349/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 2.4045e-04 - val_loss: 6.5223e-04
Epoch 350/1000
11060/11060 [==============================] - 0s 30us/sample - loss: 2.3941e-04 - val_loss: 6.4916e-04
Epoch 351/1000
11060/11060 [==============================] - 0s 27us/sample - loss: 2.3083e-04 - val_loss: 6.4633e-04
Epoch 352/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 2.2211e-04 - val_loss: 6.4362e-04
Epoch 353/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 2.1612e-04 - val_loss: 6.4100e-04
Epoch 354/1000
11060/11060 [==============================] - 0s 32us/sample - loss: 2.1695e-04 - val_loss: 6.3865e-04
Epoch 355/1000
11060/11060 [==============================] - 0s 26us/sample - loss: 2.1741e-04 - val_loss: 6.3649e-04
Epoch 356/1000
11060/11060 [====================

### Fully Connected

In [ ]:
modelType = "fullyConnected"
trainingSet, _, validationSet = getReshapedDataSetThreeSplits(dataSet, modelType)
numOfHiddenLayers = 1
codeLayerSize = 40 # 64
[model, validatoinLoss, numOfEpochs, history] = mb.trainFullyConnected(numOfHiddenLayers, codeLayerSize, trainingSet, trainingSet, validationSet, validationSet, return_best = 1)
model.save('Trained_Model/autoencoder.h5')

### 1D Conv

In [ ]:
modelType = "conv"
trainingSet, _, validationSet = getReshapedDataSetThreeSplits(dataSet, modelType)
numOfLayers = 2
filtersCountParam = 48
[model, validatoinLoss, numOfEpochs, history] = mb.trainConv(numOfLayers, trainingSet, trainingSet, validationSet, validationSet, filtersCountInFirstLayer = filtersCountParam, filterSize = 4)
#model.save('Trained_Model/conv.h5')

### LSTM 

In [ ]:
modelType = "lstm"
trainingSet, _, validationSet = getReshapedDataSetThreeSplits(dataSet, modelType)
numOfLayers = 1
numOfNeurons = 20 
[model, validatoinLoss, numOfEpochs, history] = mb.trainLstm(numOfLayers, numOfNeurons, trainingSet, trainingSet, validationSet, validationSet)
#model.save('Trained_Model/lstm.h5')